In [1]:
import numpy as np
import torch
import math 
from torch.utils.data import Dataset


def calculate_fire_spread_rate(wind_speed_at_10m, moisture_content, slope_angle, fuel_load):
    """
    Calculate the rate of spread of a forest fire.

    Parameters:
    wind_speed_at_10m (float): Wind speed at 10 meters height in km/hr.
    moisture_content (float): Moisture content in percentage (35% - 65%).
    slope_angle (float): Slope angle in degrees.
    fuel_load (float): Fuel load in tonnes/hectare (< 6 mm diameter), default is 2 tonnes/hectare.

    Returns:
    float: Rate of spread of the fire in unspecified units.
    """
    # Constants
    a = 1.674
    b = 0.1798
    c = 0.22
    d = 0.158
    e = -0.227
    f = 0.0662

    # Convert wind speed at 10m to wind speed at 1.5m
    wind_speed_at_1_5m = a + b * wind_speed_at_10m

    # Calculate the rate of spread on flat ground
    rate_flat_ground = c * fuel_load * math.exp((wind_speed_at_1_5m ** d) * (moisture_content ** e))

    # Adjust rate for slope
    rate_on_slope = rate_flat_ground * math.exp((slope_angle ** f))

    return rate_on_slope

# Example usage
spread_rate = calculate_fire_spread_rate(20, 50, 10, 2)
spread_rate

/Users/sadiahzahoor/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.4078009424508124

In [2]:
from scipy.stats import qmc
import pandas as pd


# Define the bounds for each parameter
bounds = np.array([[11.5, 34.1],  # Wind speed
                   [0, 6],        # Fuel load
                   [35, 65],      # Moisture content
                   [0, 30]])      # Slope angle

# Using Latin Hypercube Sampling (LHS) for a more structured approach
sampler = qmc.LatinHypercube(d=4)
sample = sampler.random(n=40000)
sample = qmc.scale(sample, bounds[:, 0], bounds[:, 1])

# Extracting the samples for each parameter
wind_speed_samples = sample[:, 0]
fuel_samples = sample[:, 1]
moisture_content_samples = sample[:, 2]
slope_angle_samples = sample[:, 3]

# Calculating spread rates for the sampled data
spread_rates = [calculate_fire_spread_rate(wind_speed, moisture, slope, fuel) 
                for wind_speed, moisture, slope, fuel in zip(wind_speed_samples, moisture_content_samples, slope_angle_samples, fuel_samples)]

# Creating a DataFrame to hold the sampled data and calculated spread rates
data_lhs = pd.DataFrame({
    'Wind Speed (km/hr)': wind_speed_samples,
    'Fuel Load (tonnes/hectare)': fuel_samples,
    'Moisture Content (%)': moisture_content_samples,
    'Slope Angle (degrees)': slope_angle_samples,
    'Spread Rate': spread_rates
})

data_lhs.head()

,Wind Speed (km/hr),Fuel Load (tonnes/hectare),Moisture Content (%),Slope Angle (degrees),Spread Rate
0,28.987084,0.748533,37.972195,12.095783,0.970209
1,23.637692,0.438424,41.706989,8.395726,0.538203
2,28.789305,5.080447,62.942286,3.253168,5.594417
3,33.031954,3.533723,55.175891,0.232648,3.355878
4,30.655348,0.392651,38.391963,23.075844,0.537199


from sklearn.preprocessing import MinMaxScaler

# Assuming `data_lhs` is a pandas DataFrame with your data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_lhs.iloc[:, :-1])  # Exclude the target column

# Convert the scaled data back to a DataFrame
scaled_data_lhs = pd.DataFrame(scaled_data, columns=data_lhs.columns[:-1])
scaled_data_lhs['Spread Rate'] = data_lhs['Spread Rate']  # Add the target column back


In [3]:

import torch
from torch.utils.data import Dataset

class FireSpreadDataset(Dataset):
    def __init__(self, dataframe):
        # Convert the dataframe to a PyTorch tensor
        self.data = torch.tensor(dataframe.values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Assuming the last column is the target variable (spread rate)
        features = self.data[idx, :-1]
        target = self.data[idx, -1]
        return features, target

# Convert the DataFrame to a PyTorch Dataset
fire_spread_data = FireSpreadDataset(data_lhs)

# Example: accessing the first item in the dataset
first_sample_features, first_sample_target = fire_spread_data[0]
print(first_sample_features, first_sample_target)


tensor([28.9871,  0.7485, 37.9722, 12.0958]) tensor(0.9702)


In [4]:
print(fire_spread_data[0])

(tensor([28.9871,  0.7485, 37.9722, 12.0958]), tensor(0.9702))


In [5]:
import numpy as np
import torch
from torch.utils.data import Dataset
from typing import List 
import torch.nn as nn
import torch.optim as optim

class BushfireModel(nn.Module): 
    def __init__(self, n: int, hidden_layers: List[int]): 
        super().__init__() #Inherit from the nn.module
        self.n = n # n input features
        self.hidden_layers = hidden_layers # hidden layers are the middle ones

        self.layers = nn.ModuleList([
            nn.Linear(in_dim, out_dim)
            for in_dim, out_dim in zip([n, *hidden_layers], [*hidden_layers, 1])
        ]) # Single output

    def forward(self, x: torch.tensor):
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i != len(self.layers) - 1:    #We don't want to ReLU the last layer
                x = nn.functional.relu(x)  
        return x # Linear output for regression

In [6]:
import torch
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

# Splitting the dataset
total_size = len(fire_spread_data)
train_size = int(0.8 * total_size)  # 80% for training
val_size = total_size - train_size  # 20% for validation
train_dataset, val_dataset = random_split(fire_spread_data, [train_size, val_size])

# Creating DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Training function
def train_model(model, train_loader, val_loader, loss_criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()  # Training mode
        for i, (features, labels) in enumerate(train_loader):
            # Forward pass
            outputs = model(features)
            loss = loss_criterion(outputs, labels.unsqueeze(1))

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()  # Validation mode
        with torch.no_grad():
            val_loss = 0
            for features, labels in val_loader:
                outputs = model(features)
                val_loss += loss_criterion(outputs, labels.unsqueeze(1)).item()

        val_loss /= len(val_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}')

# Initialize the model
model = BushfireModel(n=4, hidden_layers=[200,100])
loss_criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

# Train the model
num_epochs = 50
train_model(model, train_loader, val_loader, loss_criterion, optimizer, num_epochs)


Epoch [1/50], Loss: 4.5938, Val Loss: 3.9413
Epoch [2/50], Loss: 3.5458, Val Loss: 3.0092
Epoch [3/50], Loss: 1.8792, Val Loss: 2.2105
Epoch [4/50], Loss: 1.5494, Val Loss: 1.5010
Epoch [5/50], Loss: 0.9281, Val Loss: 0.9182
Epoch [6/50], Loss: 0.4784, Val Loss: 0.4934
Epoch [7/50], Loss: 0.2427, Val Loss: 0.2376
Epoch [8/50], Loss: 0.0822, Val Loss: 0.1219
Epoch [9/50], Loss: 0.0669, Val Loss: 0.0740
Epoch [10/50], Loss: 0.0347, Val Loss: 0.0490
Epoch [11/50], Loss: 0.0164, Val Loss: 0.0331
Epoch [12/50], Loss: 0.0346, Val Loss: 0.0222
Epoch [13/50], Loss: 0.0143, Val Loss: 0.0166
Epoch [14/50], Loss: 0.0155, Val Loss: 0.0133
Epoch [15/50], Loss: 0.0082, Val Loss: 0.0118
Epoch [16/50], Loss: 0.0196, Val Loss: 0.0097
Epoch [17/50], Loss: 0.0078, Val Loss: 0.0084
Epoch [18/50], Loss: 0.0050, Val Loss: 0.0079
Epoch [19/50], Loss: 0.0040, Val Loss: 0.0071
Epoch [20/50], Loss: 0.0046, Val Loss: 0.0064
Epoch [21/50], Loss: 0.0047, Val Loss: 0.0059
Epoch [22/50], Loss: 0.0032, Val Loss: 0.00

In [7]:
def predict_spread_rate(model, wind_speed, moisture_content, fuel_load, slope_angle):
    # Convert input data to tensor
    input_tensor = torch.tensor([wind_speed, moisture_content, fuel_load, slope_angle], dtype=torch.float32)

    # Reshape input_tensor to match the input shape of the model
    input_tensor = input_tensor.unsqueeze(0)  # Adds a batch dimension

    # Set the model to evaluation mode
    model.eval()

    # Perform inference
    with torch.no_grad():
        prediction = model(input_tensor)

    # Return the predicted spread rate
    return prediction.item()

# Example usage
predicted_spread_rate = predict_spread_rate(model, 30, 40, 0, 2)
print("Predicted Spread Rate:", predicted_spread_rate)

# Example usage
spread_rate = calculate_fire_spread_rate(30, 40, 0, 2)
print("Actual Spread Rate:", spread_rate)


Predicted Spread Rate: 22.859973907470703
Actual Spread Rate: 0.7933984307219112


In [8]:
def predict(model, input_features):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        predictions = model(input_features)
    return predictions

# Example usage with one sample from the validation set
features, labels = next(iter(val_loader))
prediction = predict(model, features)
print(f'Predicted: {prediction}, Actual: {labels}')


Predicted: tensor([[6.5694],
        [1.5881],
        [6.7419],
        [0.6962],
        [5.4724],
        [1.0508],
        [1.3484],
        [1.7695],
        [3.5320],
        [5.2887],
        [5.4050],
        [7.1798],
        [2.5160],
        [3.0784],
        [1.9516],
        [0.8473],
        [6.5617],
        [1.8287],
        [5.6586],
        [4.6032],
        [3.5750],
        [2.2047],
        [7.7249],
        [4.8136],
        [7.3112],
        [3.9456],
        [4.2175],
        [5.5604],
        [0.3689],
        [3.2668],
        [7.6154],
        [4.8626],
        [4.8139],
        [5.5875],
        [7.5043],
        [3.5340],
        [4.3038],
        [2.2063],
        [1.2744],
        [1.0070],
        [2.5820],
        [4.6170],
        [0.5150],
        [4.3200],
        [3.0408],
        [6.3921],
        [7.4744],
        [6.4214],
        [0.7390],
        [6.1311],
        [1.2211],
        [3.7842],
        [6.3518],
        [3.3541],
        [4.3273],